# Use RNN to predict sudoku Sequence

## dataset 
- [10,000 solved sudoku](http://www.printable-sudoku-puzzles.com/wfiles/)

## Load sudoku training data

In [76]:
import numpy as np
from glob import glob

import torch
from torch.autograd import Variable
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

In [77]:

puzzles = sum([open(f).readlines() 
               for f in glob("/home/dola/data/sudoku/solved/*.txt")], [])
puzzles = [p.strip() for p in puzzles if len(p.strip())==81]
len(puzzles)

10000

In [78]:
puzzles[0]

'123456789578139624496872153952381467641297835387564291719623548864915372235748916'

### check they are valid sudoku

In [35]:
def check_sudoku(puzzle):
    assert len(puzzle) == 81
    p = np.array(list(puzzle)).reshape(9, 9)
    rows = range(9)
    cols = range(9)
    digits = set('123456789')
    strides = [slice(0, 3), slice(3, 6), slice(6, 9)]
    squares = [(r, c) for r in strides for c in strides]
    for r in rows:
        assert set(p[r,:]) == digits, "err: row %i" % r
    for c in cols:
        assert set(p[:,c]) == digits, "err: col %i" % c
    for sr, sc in squares:
        assert set(p[sr, sc].ravel()) == digits, "err: sqr %i %i" % (sr, sc)

In [36]:
# negative example
p = ''.join(['123456789'] * 9)
print(p)
check_sudoku(p)

123456789123456789123456789123456789123456789123456789123456789123456789123456789


AssertionError: err: col 0

In [37]:
for p in puzzles:
    check_sudoku(p)

## Data Processing

### data augumentation

In [47]:
from tqdm import tqdm_notebook

In [79]:
%%time
def augument(puzzles):
    augumented = [p for p in puzzles]
#     augumented += [p[::-1] for p in puzzles]
    for p in puzzles:
        p = np.array(list(p)).reshape([9, 9])
        augumented.append(''.join(np.fliplr(p).ravel()))
        p = p.T
        augumented.append(''.join(p.ravel()))
        augumented.append(''.join(np.fliplr(p).ravel()))
        p = p.T
        augumented.append(''.join(p.ravel()))
        augumented.append(''.join(np.fliplr(p).ravel()))
        p = p.T
        augumented.append(''.join(p.ravel()))
        augumented.append(''.join(np.fliplr(p).ravel()))
    return augumented

puzzles = augument(puzzles)

CPU times: user 1.64 s, sys: 92 ms, total: 1.73 s
Wall time: 1.61 s


In [82]:
print(len(puzzles))

for p in puzzles:
    check_sudoku(p)

80000


In [83]:
symbol2index = dict(zip('123456789', range(9)))
index2symbol = dict(zip(range(9), '123456789'))

def puzzle2tensor(puzzle_batch):
    batch_size = len(puzzle_batch)
    seq_len = len(puzzle_batch[0])
    t = torch.zeros([batch_size, seq_len, 9])
    for r in range(batch_size):
        for c in range(seq_len):
            s = symbol2index[puzzle_batch[r][c]]
            t[r, c, s] = 1
    return t

def puzzle2target(puzzle_batch):
    batch_size = len(puzzle_batch)
    seq_len = len(puzzle_batch[0])
    t = torch.LongTensor(batch_size, seq_len).zero_()
    for r in range(batch_size):
        for c in range(seq_len):
            s = symbol2index[puzzle_batch[r][c]]
            t[r, c] = s
    return t

def tensor2puzzle(tensors):
    """tensors.size() == [batch_size, seq, 9]
    """
    _, p = tensors.max(dim=2)
    p = p.squeeze().numpy()
    puzzles = []
    for r in p:
        puzzles.append(''.join([index2symbol.get(s) for s in r]))
    return puzzles

def output2puzzle(y):
    """y.size() == [batch_size, 9]
    """
    _, labels = y.max(dim=1)
    return labels.numpy().squeeze() + 1

In [84]:
## test
t = puzzle2tensor([p[:5] for p in puzzles[:2]])
p = tensor2puzzle(t)
print(t.size())
print(p)

torch.Size([2, 5, 9])
['12345', '12345']


In [85]:
## test
puzzle2target([p[:5] for p in puzzles[:2]])


 0  1  2  3  4
 0  1  2  3  4
[torch.LongTensor of size 2x5]

## Models - seq prediction 

In [103]:
class SeqModel(nn.Module):
    def __init__(self):
        super(SeqModel, self).__init__()
        self.input_size = 9
        self.rnn_layers = 2
        self.rnn_hidden_size = 64
        self.fc1_hidden_size = 128
        self.fc2_hidden_size = 128
        self.output_size = 9
        self.rnn = nn.GRU(input_size=self.input_size, dropout=0.5,
                          hidden_size=self.rnn_hidden_size,
                          num_layers=self.rnn_layers,
                          batch_first=True,
                          bidirectional=False)
        self.fc1 = nn.Linear(self.rnn_hidden_size, self.fc1_hidden_size)
        self.elu = nn.ELU()
        self.fc2 = nn.Linear(self.fc1_hidden_size, self.fc2_hidden_size)
        self.bn2 = nn.BatchNorm1d(self.fc2_hidden_size)
        self.fc3 = nn.Linear(self.fc2_hidden_size, self.output_size)
#         self.softmax = nn.Softmax()
        
#         self.fc1.weight.data.uniform_(-0.1, 0.1)
#         self.fc2.weight.data.uniform_(-0.1, 0.1)
#         self.fc3.weight.data.uniform_(-0.1, 0.1)
    def forward(self, x, h0=None):
        batch_size = x.size(0)
        seq_len = x.size(1)
        input_size = x.size(2)
        
        if h0 is None:
            h0 = Variable(torch.zeros(self.rnn_layers, batch_size, self.rnn_hidden_size)).cuda()
        out, h = self.rnn(x, h0)
        
        out = out[:, -1, :]
        out = self.elu(self.fc1(out))
        out = self.elu(self.fc2(out))
        out = self.bn2(out)
        out = self.fc3(out)
#         out = self.softmax(self.fc3(out))
#         out = out.view([batch_size, self.output_size])
        return out, h
        
seq_model = SeqModel().cuda()

In [104]:
## test

# x = Variable(torch.rand(30, seq_model.seq_len, seq_model.input_size)).cuda()
x = Variable(puzzle2tensor([p[:5] for p in puzzles[100:120]])).cuda()
y, h = seq_model(x)
y.size(), h.size()

(torch.Size([20, 9]), torch.Size([2, 20, 64]))

In [105]:
h0 = h.detach()
y, h = seq_model(x, h0)
y.size(), h.size()

(torch.Size([20, 9]), torch.Size([2, 20, 64]))

In [106]:
output2puzzle(y.cpu().data)

array([6, 3, 3, 4, 2, 6, 5, 9, 1, 9, 3, 1, 3, 6, 2, 9, 4, 2, 2, 3])

## training

In [107]:
objective = nn.CrossEntropyLoss()

puzzles = np.array(puzzles)


In [127]:
seq_model = SeqModel().cuda()
seq_model.train()
optimizer = optim.Adam(seq_model.parameters(), lr=1e-4)

n_epochs = 10
batch_size = 128
n = len(puzzles)
n_batches = n // batch_size
puzzle_len = len(puzzles[0])
window = 64

for epoch in range(n_epochs):
    random_index = np.random.permutation(n)
    for b, i in enumerate(np.array_split(random_index, n_batches)):
        puzzle_batch = puzzles[i]
        puzzle_tensor = puzzle2tensor(puzzle_batch)
        puzzle_target = puzzle2target(puzzle_batch)
        h0 = None
        for w in range(0, puzzle_len-window):
            input_tensor = puzzle_tensor[:, w:w+window, :]
            output_tensor = puzzle_target[:, w+window]
            x = Variable(input_tensor).cuda()
            y = Variable(output_tensor).cuda()
            
            seq_model.zero_grad()
            
            yhat, h = seq_model(x, h0)
            h0 = h.detach()
            loss = objective(yhat, y)
            loss.backward()
            optimizer.step()
            
        if b % 100 == 0:
            print(epoch, b, loss.data[0])

0 0 2.246368169784546
0 100 0.8028936982154846
0 200 0.4710676074028015
0 300 0.35784584283828735
0 400 0.24215717613697052
0 500 0.24460119009017944
0 600 0.22324861586093903
1 0 0.21908637881278992
1 100 0.18274909257888794
1 200 0.17952166497707367
1 300 0.1439046561717987
1 400 0.1632360816001892
1 500 0.14002855122089386
1 600 0.14462237060070038
2 0 0.17436811327934265
2 100 0.09254685044288635
2 200 0.0896424651145935
2 300 0.12730944156646729
2 400 0.13667356967926025
2 500 0.13280457258224487
2 600 0.08418458700180054
3 0 0.10701041668653488
3 100 0.10469381511211395
3 200 0.10124282538890839
3 300 0.07951025664806366
3 400 0.09279587864875793
3 500 0.06239498406648636
3 600 0.1037537157535553
4 0 0.09408758580684662
4 100 0.07682383060455322
4 200 0.05804728716611862
4 300 0.05847519263625145
4 400 0.08347370475530624
4 500 0.07094751298427582
4 600 0.064023956656456
5 0 0.06047211214900017
5 100 0.04231695458292961
5 200 0.06333222985267639
5 300 0.06712163984775543
5 400 0.

KeyboardInterrupt: 

In [109]:
def generate_puzzle(seeds, model):
    """seeds.size() == [batch_size, window]
    """
    model.eval()
    
    batch_size = len(seeds)
    window = len(seeds[0])
    
    generated = np.array([['x'] * 81] * batch_size)
    generated[:, :window] = np.array([list(r) for r in seeds])
    h0 = None
    for i in range(0, 81-window):
        x = Variable(puzzle2tensor([p[i:i+window] for p in generated])).cuda()
        y, h = model(x, h0)
        h0 = h.detach()
        y = output2puzzle(y.cpu().data)
        generated[:, i+window]  = y
    return ["".join(r) for r in generated]
    

In [128]:
generated = generate_puzzle([p[:window] for p in puzzles[100:105]], seq_model)
generated

['123456789689713254547829136234678915961542378758391642312984567476295138587316249',
 '123456789749183265865297134458319627291675843376842591912564378674831592591823467',
 '123456789746198253859723641371842596285679134694531872462387915517293466878431299',
 '123456789694378521578129643217845396469231857385697214851963472742136591527643878',
 '123456789948713256576892314861947532294385167735261948619524873387421596695843172']

In [130]:
check_sudoku(generated[0])

AssertionError: err: col 0

In [132]:
np.array(list(generated[0])).reshape([9, 9])

array([['1', '2', '3', '4', '5', '6', '7', '8', '9'],
       ['6', '8', '9', '7', '1', '3', '2', '5', '4'],
       ['5', '4', '7', '8', '2', '9', '1', '3', '6'],
       ['2', '3', '4', '6', '7', '8', '9', '1', '5'],
       ['9', '6', '1', '5', '4', '2', '3', '7', '8'],
       ['7', '5', '8', '3', '9', '1', '6', '4', '2'],
       ['3', '1', '2', '9', '8', '4', '5', '6', '7'],
       ['4', '7', '6', '2', '9', '5', '1', '3', '8'],
       ['5', '8', '7', '3', '1', '6', '2', '4', '9']], 
      dtype='<U1')